## PDI - Phuture DeFi Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyProd
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "decentralized-finance-defi"

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyProd(
                                1,
                                'ethereum',
                                '0x632806BF5c8f062932Dd121244c9fbe7becb8B48',
                                min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    'ethereum',
    df_to_remove=[stablecoins,ctokens,atokens],
    add_category_assets=['yearn-finance','convex-finance','balancer','compound-governance-token'],
    remove_category_assets=['the-graph','staked-ether','injective-protocol']
    )
liquidity

,spot price,delivery price,slippage,blockchain
id,,,,
convex-finance,0.239840,0.240701,0.003588,ethereum
aave,0.010280,0.010316,0.003538,ethereum
maker,0.000691,0.000693,0.001839,ethereum
chainlink,0.059835,0.059817,-0.000299,ethereum
lido-dao,0.413086,0.411352,-0.004197,ethereum
rocket-pool,0.031923,0.031720,-0.006334,ethereum
uniswap,0.157492,0.155218,-0.014437,ethereum
frax-share,0.111924,0.109837,-0.018643,ethereum
curve-dao-token,1.481503,1.447302,-0.023085,ethereum


## Show Results

In [4]:
results

,name,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
chainlink,Chainlink,9267291220,0.300000,77,0x514910771af9ca656af840dff83e8264ecf986ca,ethereum
uniswap,Uniswap,4829261994,0.250160,64,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,ethereum
lido-dao,Lido DAO,2150230245,0.111384,29,0x5a98fcbea516cf06857215779fd812ca3bef1b32,ethereum
aave,Aave,1418921576,0.073501,19,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,ethereum
maker,Maker,1317940332,0.068271,18,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,ethereum
havven,Synthetix Network,1237266142,0.064092,17,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,ethereum
frax-share,Frax Share,669746314,0.034693,8,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,ethereum
curve-dao-token,Curve DAO,637927013,0.033045,8,0xd533a949740bb3306d119cc777fa900ba034cd52,ethereum
rocket-pool,Rocket Pool,620693200,0.032152,8,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,ethereum


## Contract Output

In [5]:
methodology_class.output_for_contract()

0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,0x514910771af9ca656af840dff83e8264ecf986ca,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,0xba100000625a3754423978a60c9317c58a424e3d,0xc00e94cb662c3520282e6f5717214004a7f26888,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,0xd533a949740bb3306d119cc777fa900ba034cd52
3,64,8,4,77,29,19,18,0,0,17,8,8
